# PyTorch convolutional layers

In [65]:
import numpy as np
import torch

x = np.array([
    [1, 2, 3, 4, 5],
    [6, 7, 8 ,9, 10],
    [11, 12, 13, 14, 15]
])

k = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])


# Define convolution function
def convo2d(input, kernel):
    H,W = input.shape
    print(f"H: {H}", f"W: {W}")
    M,N = kernel.shape
    print(f"M: {M}", f"N: {N}")
    out = np.zeros((H-M+1,W-N+1), dtype=float)
    kernel = np.flip(kernel)
    for i in range(H-M+1):
        for j in range(W-N+1):
            out[i,j] = np.sum( input[i:i+M,j:j+N] * kernel)
    return out

o = convo2d(x, k)
print(o)

H: 3 W: 5
M: 3 N: 3
[[219. 264. 309.]]


In [71]:
# Create 8 x 8 tensor with half 4s, half 1s
arr = np.zeros((8, 8), dtype=float)
arr[:,:4] = 4.0
arr[:,4:] = 1.0
print("arr:", arr)

# Create 3 x 3 kernel with [-1, 0, 1] for each row
ker = np.zeros((3, 3), dtype=float)
ker[:,:1] = -1.0
ker[:,2:] = 1.0
print("ker:", ker)

# Make into tensor and unsqueeze to change shape from (8, 8) to (1, 1, 8, 8) for input to match 
# (batch_size, num_input_channels, image_height, image_width)
arr = torch.from_numpy(arr)
arr = torch.unsqueeze(arr, 0)
arr = torch.unsqueeze(arr, 0)

# Do the same for the kernel to change shape from (3, 3) to (1, 1, 3, 3) to match
# (out_channels, in_channels, kernel_height, kernel_width)
ker = torch.from_numpy(ker)
ker = torch.unsqueeze(ker, 0)
ker = torch.unsqueeze(ker, 0)

out = torch.nn.functional.conv2d(arr, ker, stride=1)
print("out:", out)
print("out.shape:", out.shape)

arr: [[4. 4. 4. 4. 1. 1. 1. 1.]
 [4. 4. 4. 4. 1. 1. 1. 1.]
 [4. 4. 4. 4. 1. 1. 1. 1.]
 [4. 4. 4. 4. 1. 1. 1. 1.]
 [4. 4. 4. 4. 1. 1. 1. 1.]
 [4. 4. 4. 4. 1. 1. 1. 1.]
 [4. 4. 4. 4. 1. 1. 1. 1.]
 [4. 4. 4. 4. 1. 1. 1. 1.]]
ker: [[-1.  0.  1.]
 [-1.  0.  1.]
 [-1.  0.  1.]]
out: tensor([[[[ 0.,  0., -9., -9.,  0.,  0.],
          [ 0.,  0., -9., -9.,  0.,  0.],
          [ 0.,  0., -9., -9.,  0.,  0.],
          [ 0.,  0., -9., -9.,  0.,  0.],
          [ 0.,  0., -9., -9.,  0.,  0.],
          [ 0.,  0., -9., -9.,  0.,  0.]]]], dtype=torch.float64)
out.shape: torch.Size([1, 1, 6, 6])


## Now with classes instead of functions

In [87]:
from torch import nn
# Required args for Conv2d class are (in_channels, out_channels, kernel_size)
# Kernel size can be scalar for square kernels, tuple for nonsquare
# Input tensor must have 4 axes: (batch_size, in_channels, H, W)

# Create Convolution 2d layer with 1 in_channel, 1 out_channel, and kernel_size 3 respectively
conop = nn.Conv2d(1, 1, 3, bias=False) ## assumes a stride of 1

# Random 1 x 1 x 8 x 8 tensor (remember that's 4 axes but 1 * 1 * 8 * 8 = 64 dimensions per sample)
input = torch.randn(1, 1, 8, 8)
print(input)
print(input.shape)
print(input.type())

tensor([[[[ 0.0618, -0.3360, -1.4161,  0.1578,  0.3603, -1.1822, -0.9412,
            0.6454],
          [ 0.5408, -1.7552, -0.7583, -0.2435,  1.3099,  1.8129, -0.2258,
            0.1054],
          [-1.5631,  0.1750, -0.1093,  1.2586,  0.8260,  0.9647,  0.3008,
           -1.4056],
          [-0.5501,  0.5267,  1.3969, -1.2911, -0.0650,  0.2992,  0.4321,
           -2.1099],
          [-0.3377, -0.3679, -0.5889, -0.9092,  0.6957, -0.4958,  0.5461,
            0.2581],
          [-0.8905, -1.2597,  1.0387,  0.0769, -0.4658, -0.7012,  0.3702,
            0.0983],
          [-0.4355,  0.3719, -0.8343,  0.3197, -0.6816,  0.9041,  1.7628,
           -1.3260],
          [ 0.2247, -1.4334, -0.4431,  0.4050, -1.2078, -1.5878, -0.1956,
            2.2635]]]])
torch.Size([1, 1, 8, 8])
torch.FloatTensor


In [89]:
# Pass input through Conv2d layer
output = conop(input)

print(output)
print(output.shape)
print(output.type())

tensor([[[[-0.1636,  0.3696, -0.4037,  0.0316,  0.3237, -0.2373],
          [ 0.8393,  0.1395,  0.0454, -0.7921, -0.4253,  0.7620],
          [ 0.0601,  0.3040, -1.2482,  0.5007, -0.2265,  0.0976],
          [-0.6007, -0.2193,  0.8332,  0.2552, -0.4785,  0.3211],
          [ 0.3870,  0.5986,  0.0270, -0.7675, -0.1456,  0.2733],
          [ 0.3963, -0.8957,  0.8574,  0.4402,  0.6020, -0.8086]]]],
       grad_fn=<ThnnConv2DBackward>)
torch.Size([1, 1, 6, 6])
torch.FloatTensor


In [100]:
# We can examine the learned kernel weights
ker = conop.weight

print(ker)
print(ker.shape)
print(ker.type())

Parameter containing:
tensor([[[[ 0.2275, -0.3114, -0.0669],
          [-0.2838,  0.1715,  0.0298],
          [-0.0457, -0.0035, -0.2681]]]], requires_grad=True)
torch.Size([1, 1, 3, 3])
torch.FloatTensor
